<a href="https://colab.research.google.com/github/bennyqp/artificial-inspiration/blob/main/Image%20Generation/ai_image_generation_SinGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SinGAN
_

##This is a notebook from Derrick Schulz
This notebook is completely and with no changes copied from Derrick Schulz

(https://github.com/dvschultz)

You can find the original here: 

https://github.com/dvschultz/ai/blob/master/SinGAN.ipynb

_

SinGAN is a ML library that allows you to do a lot of different things with one image as your dataset. As we’ve previously discussed usually you need a ton of images to be able to do anything in ML, so SinGAN needing only one image is a huge change.

It is, however, important to keep in mind what its outputs are. SinGAN, to me, is most like Photoshop’s Context Aware tools. It allows you to manipulate the single image in interesting ways, but in the end is only going to return you that same image, slightly modified. SinGAN is a bit of a Swiss Army Knife and that makes it pretty cool in a world of single-use ML tools.

Let’s take a look.

##Set up our Runtime
Colab needs to know we need to use a GPU-powered machine in order to do style transfers. At the top of this page, click on the `Runtime` tab, then select `Change runtime type`. In the modal that pops up, select `GPU` under the `Hardware accelerator` options.

In [ ]:
!nvidia-smi -L

## Set up SinGAN
Let’s install the SinGAN repo and install all the dependencies needed

In [ ]:
!git clone https://github.com/dvschultz/SinGAN
!pip install torch==1.4.0 torchvision==0.5.0
%cd /content/SinGAN/

We now have SinGAN installed on CoLab. Here are some relevant links to understand how to use the library:

*   [SinGAN Github repo](https://github.com/tamarott/SinGAN)
*   [ArXiv paper](https://arxiv.org/pdf/1905.01164.pdf)
* [Supplementary Materials paper](https://tomer.net.technion.ac.il/files/2019/09/SingleImageGan_SM.pdf)





##Training your image
In order to use SinGAN, we need to train the model on our image. In my experience this usually takes an hour or so to train a 250px image.

The `main_train.py` script takes one required argument, `--input_name`. To use with a custom image, upload an image to the `Input/Images` folder and then pass its name and file extension to the argument. 

In [ ]:
!python main_train.py --input_name colusseum.png --max_size 256

##Generating Random Samples
By default, once training is completed 50 samples are generated. That means you don’t need to run the next line of code, unless you want to change some of the default options. 

Note: If you do run the next line of code, you’ll need to delete the folder of output images that were created during training. Otherwise you will get an error.

###Sample Options

####Number of Images
`--num_samples` (note: this only works in my version of the library)

You can pass in a number to generate any number of images. By default it generates 50. I recommend using this when running the `main_train.py` script otherwise you’ll need to delete the output folder and re-run it.

####Generator scale
`--gen_start_scale` (pass is a number, usually between 0–3)

I don’t want to dig into this too deeply, so I’ll just say the generator scale relates to how "realistic" the new images look. If you use `0` you will get images that differ drastically from your original image. If you use `2` or `3` the differences will be very subtle. Anything above these numbers will give you almost no changes.

####Mode
`--mode`
There are two. While the default `random_samples` generates images of the same size as your training image, `random_samples_arbitrary_sizes` changes the size of your canvas. It’s not a scaled up image, however, its more like it makes the canvas larger and tries to fill the space added with a similar texture.

####Scale of canvas
`--scale_h` (pass it a floating number)

`--scale_v` 

For use with `random_samples_arbitrary_sizes`, this allows you to define how much larger (or smaller) the canvas should be. you should pass it a floating value that scales each dimension up or down (`1.0` is the same size, `2.0` is twice as large, etc).

In [ ]:
# example using the default random_samples mode
!python random_samples.py --input_name colusseum.png --mode random_samples --gen_start_scale 1 --num_samples 10 --max_size 256

Random Seed:  2542


In [ ]:
# example using the random_samples_arbitrary_sizes mode
# can only run at --gen_start_scale 0 (https://github.com/tamarott/SinGAN/issues/61)
!python random_samples.py --input_name colusseum.png --mode random_samples_arbitrary_sizes --scale_h 2.0 --scale_v 1.0 --gen_start_scale 0 --num_samples 10

Random Seed:  756


##Generating Animations
SinGAN can generate an animation from the single image. It essentially create a noisy peturbation on the image causing it to animate.

Unfortunately, this requires another training session on the image. It should take the same amount of time as your previous training.

`--min_size` and `--max_size` can be used here as well.

In [ ]:
!python animation.py --input_name colusseum.png --max_size 256

You can play the video in Colab by pointing the path to the following function (you may need to update the width and height arguments as well):

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/SinGAN/Output/Animation/colusseum/start_scale=2/alpha=0.100000_beta=0.800000.mp4', width=256, height=192)

###Animation Options

####Output type
`--output_type`

This allows you to output either `video` or `gif`. The default is `video`.

`--num_frames`

This determines the length of your animation. the default is `100` frames (10fps, so 10 seconds)

In [ ]:
!python animation.py --input_name colusseum.png --max_size 256 --output_type gif --num_frames 50